# Gender Classificaton


## Imports

In [7]:
import os
import sys
from scrape import get_remote_tgz_files, download_extract_files
from parse import paths, parse_readme
from pathlib import Path
import sqlite3

## Input variables

In [8]:
SOURCE_URL = "http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/"
DATA_DIR = "data"
DB = ""